In [335]:
import numpy as np
import random
from scipy.special import softmax

In [336]:
values = np.zeros((3**9, ))

In [337]:
def serialize(board):
    serial = 0
    for i in range(9):
        serial = serial + board[i]*3**(i)
    return int(serial)

In [338]:
def print_board(serial):
    board = np.zeros((9,))
    for i in range(9):
        board[i] = serial%3
        serial = serial/3
    print(board[0:3])
    print(board[3:6])
    print(board[6:9])

In [340]:
class game:
    def __init__(self, training):
        self.board = np.zeros((9,))
        self.num_moves = 0
        self.history = []
        self.training = training
        self.player = int(np.random.uniform()>0.5)
        
        
    def clear_history(self):
        self.board = np.zeros((9,))
        self.history = []
        self.num_moves = 0
        
    def make_move(self, move):
        self.board[move] = self.num_moves%2+1
        self.history += [serialize(self.board)]
        self.num_moves = self.num_moves+1
        
    def game_won(self):
        if (self.board[0] == self.board[1] and self.board[0] == self.board[2] and self.board[0] != 0) or \
            (self.board[0] == self.board[4] and self.board[0] == self.board[8] and self.board[0] != 0) or \
            (self.board[0] == self.board[3] and self.board[0] == self.board[6] and self.board[0] != 0) or \
            (self.board[1] == self.board[4] and self.board[1] == self.board[7] and self.board[1] != 0) or \
            (self.board[2] == self.board[4] and self.board[2] == self.board[6] and self.board[2] != 0) or \
            (self.board[2] == self.board[5] and self.board[2] == self.board[8] and self.board[2] != 0) or \
            (self.board[3] == self.board[4] and self.board[3] == self.board[5] and self.board[3] != 0) or \
            (self.board[6] == self.board[7] and self.board[6] == self.board[8] and self.board[6] != 0):
            return True
        return False
    
    def game_over(self):
        if self.game_won():
            if self.num_moves%2 == 1:
                return 1
            else:
                return 2
        else:
            return 0
    
    def legal_moves(self):
        moves = []
        for i in range(9):
            if self.board[i] == 0:
                moves += [i]
        return moves
    
    def get_move(self):
        moves = self.legal_moves()
        vs = np.zeros((len(moves),))
        potential_boards = [self.board.copy() for i in range(len(moves))]
        for ind, m in enumerate(moves):
            potential_boards[ind][m] = self.num_moves%2+1
            vs[ind] = -values[serialize(potential_boards[ind])]
            if not self.training:
                potential_boards[ind].print_game()
                

        if self.training:
            return moves[np.argmax(np.random.multinomial(1, softmax(vs)))]
        else:
            return moves[np.argmax(vs)]
    
    def update_values(self):
        winner = 2-(self.num_moves%2) # 1 if p1 won, 2 if p2 won
        for ind, b in enumerate(self.history):
            values[b] = values[b]+(-1+2*((ind-winner)%2))/(ind+1.)
            #print(values[b])
            
    def print_game(self):
        print(self.board[0:3])
        print(self.board[3:6])
        print(self.board[6:9])


In [341]:
def play_game():
    training_game = game(1)
    while training_game.num_moves < 9 and training_game.game_over() == 0:
        training_game.make_move(training_game.get_move())
    #training_game.print_game()
    if training_game.game_won():
        training_game.update_values()
    #training_game.clear_history()
    return training_game.game_over()

In [342]:
for i in range(1000):
    winner = play_game()
#     if winner == 1:
#         print('p1 won')
#     elif winner == 2:
#         print('p2 won')
#     else:
#         print('draw')

In [307]:
bot_turn = 1
actual_game = game(0)
turn = 0
while not actual_game.game_over():
    if not turn == bot_turn:
        move = input("play_move: ")
        actual_game.make_move(move)
    else:
        actual_game.make_move(actual_game.get_move())
    actual_game.print_game()
    turn = 1-turn

    
if actual_game.game_over() == bot_turn+1:
    print('you lost')
if actual_game.game_over() == 2-bot_turn:
    print('you won')
elif actual_game.game_over() == 3:
    print('tie')
    

play_move: 0
[1. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[1. 0. 2.]
[0. 0. 0.]
[0. 0. 0.]
play_move: 8
[1. 0. 2.]
[0. 0. 0.]
[0. 0. 1.]
[1. 0. 2.]
[0. 2. 0.]
[0. 0. 1.]
play_move: 6
[1. 0. 2.]
[0. 2. 0.]
[1. 0. 1.]
[1. 0. 2.]
[0. 2. 0.]
[1. 2. 1.]
play_move: 3
[1. 0. 2.]
[1. 2. 0.]
[1. 2. 1.]
you won


In [343]:
for v in range(3**9):
    print_board(v)
    print(values[v])

[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[1. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
31.0
[2. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[0. 1. 0.]
[0. 0. 0.]
[0. 0. 0.]
32.0
[1. 1. 0.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[2. 1. 0.]
[0. 0. 0.]
[0. 0. 0.]
-0.5
[0. 2. 0.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[1. 2. 0.]
[0. 0. 0.]
[0. 0. 0.]
-3.5
[2. 2. 0.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[0. 0. 1.]
[0. 0. 0.]
[0. 0. 0.]
31.0
[1. 0. 1.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[2. 0. 1.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[0. 1. 1.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[1. 1. 1.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[2. 1. 1.]
[0. 0. 0.]
[0. 0. 0.]
-0.6666666666666666
[0. 2. 1.]
[0. 0. 0.]
[0. 0. 0.]
-7.5
[1. 2. 1.]
[0. 0. 0.]
[0. 0. 0.]
0.6666666666666666
[2. 2. 1.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[0. 0. 2.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[1. 0. 2.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[2. 0. 2.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[0. 1. 2.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[1. 1. 2.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[2. 1. 2.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[0. 2. 2.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[1. 2. 2.]
[0. 0. 0.]
[0. 0. 0.]
0.0
[

[0. 1. 0.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[1. 1. 0.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[2. 1. 0.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[0. 2. 0.]
[1. 0. 1.]
[0. 0. 0.]
1.3333333333333333
[1. 2. 0.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[2. 2. 0.]
[1. 0. 1.]
[0. 0. 0.]
0.25
[0. 0. 1.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[1. 0. 1.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[2. 0. 1.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[0. 1. 1.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[1. 1. 1.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[2. 1. 1.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[0. 2. 1.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[1. 2. 1.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[2. 2. 1.]
[1. 0. 1.]
[0. 0. 0.]
0.2
[0. 0. 2.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[1. 0. 2.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[2. 0. 2.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[0. 1. 2.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[1. 1. 2.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[2. 1. 2.]
[1. 0. 1.]
[0. 0. 0.]
0.4
[0. 2. 2.]
[1. 0. 1.]
[0. 0. 0.]
-0.25
[1. 2. 2.]
[1. 0. 1.]
[0. 0. 0.]
0.2
[2. 2. 2.]
[1. 0. 1.]
[0. 0. 0.]
0.0
[0. 0. 0.]
[2. 0. 1.]
[0. 0. 0.]
0.5
[1. 0. 0.]
[2. 0. 1.]
[0. 0. 0.]
-0.3333333333333333
[2. 

[0. 0. 0.]
0.0
[1. 2. 2.]
[2. 0. 2.]
[0. 0. 0.]
0.0
[2. 2. 2.]
[2. 0. 2.]
[0. 0. 0.]
0.0
[0. 0. 0.]
[0. 1. 2.]
[0. 0. 0.]
-11.5
[1. 0. 0.]
[0. 1. 2.]
[0. 0. 0.]
1.0
[2. 0. 0.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[0. 1. 0.]
[0. 1. 2.]
[0. 0. 0.]
2.333333333333333
[1. 1. 0.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[2. 1. 0.]
[0. 1. 2.]
[0. 0. 0.]
-0.5
[0. 2. 0.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[1. 2. 0.]
[0. 1. 2.]
[0. 0. 0.]
-0.75
[2. 2. 0.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[0. 0. 1.]
[0. 1. 2.]
[0. 0. 0.]
1.3333333333333333
[1. 0. 1.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[2. 0. 1.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[0. 1. 1.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[1. 1. 1.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[2. 1. 1.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[0. 2. 1.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[1. 2. 1.]
[0. 1. 2.]
[0. 0. 0.]
0.2
[2. 2. 1.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[0. 0. 2.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[1. 0. 2.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[2. 0. 2.]
[0. 1. 2.]
[0. 0. 0.]
0.0
[0. 1. 2.]
[0. 1. 2.]
[0. 0. 0.]
-0.25
[1. 1. 2.]
[0. 1. 2.]
[0. 0. 0.]
0.4
[2. 1. 2.]
[0. 1. 2.]
[0

[1. 0. 0.]
-0.25
[0. 1. 0.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[1. 1. 0.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[2. 1. 0.]
[2. 1. 0.]
[1. 0. 0.]
0.2
[0. 2. 0.]
[2. 1. 0.]
[1. 0. 0.]
-0.25
[1. 2. 0.]
[2. 1. 0.]
[1. 0. 0.]
0.2
[2. 2. 0.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[0. 0. 1.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[1. 0. 1.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[2. 0. 1.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[0. 1. 1.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[1. 1. 1.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[2. 1. 1.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[0. 2. 1.]
[2. 1. 0.]
[1. 0. 0.]
0.2
[1. 2. 1.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[2. 2. 1.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[0. 0. 2.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[1. 0. 2.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[2. 0. 2.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[0. 1. 2.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[1. 1. 2.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[2. 1. 2.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[0. 2. 2.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[1. 2. 2.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[2. 2. 2.]
[2. 1. 0.]
[1. 0. 0.]
0.0
[0. 0. 0.]
[0. 2. 0.]
[1. 0. 0.]
0.0
[1. 0. 0.]
[0. 2. 0.]
[1. 0. 0.]
0.6666666666666666
[2. 

[2. 1. 1.]
[1. 0. 0.]
0.0
[1. 1. 1.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[2. 1. 1.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[0. 2. 1.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[1. 2. 1.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[2. 2. 1.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[0. 0. 2.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[1. 0. 2.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[2. 0. 2.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[0. 1. 2.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[1. 1. 2.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[2. 1. 2.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[0. 2. 2.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[1. 2. 2.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[2. 2. 2.]
[2. 1. 1.]
[1. 0. 0.]
0.0
[0. 0. 0.]
[0. 2. 1.]
[1. 0. 0.]
0.3333333333333333
[1. 0. 0.]
[0. 2. 1.]
[1. 0. 0.]
0.0
[2. 0. 0.]
[0. 2. 1.]
[1. 0. 0.]
0.0
[0. 1. 0.]
[0. 2. 1.]
[1. 0. 0.]
0.0
[1. 1. 0.]
[0. 2. 1.]
[1. 0. 0.]
0.0
[2. 1. 0.]
[0. 2. 1.]
[1. 0. 0.]
0.0
[0. 2. 0.]
[0. 2. 1.]
[1. 0. 0.]
0.0
[1. 2. 0.]
[0. 2. 1.]
[1. 0. 0.]
-0.2
[2. 2. 0.]
[0. 2. 1.]
[1. 0. 0.]
0.0
[0. 0. 1.]
[0. 2. 1.]
[1. 0. 0.]
0.0
[1. 0. 1.]
[0. 2. 1.]
[1. 0. 0.]
0.0
[2. 0. 1.]
[0. 2. 1.]
[1. 0. 0.]


...

[0. 1. 0.]
[1. 2. 2.]
0.0
[0. 0. 1.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[1. 0. 1.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[2. 0. 1.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[0. 1. 1.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[1. 1. 1.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[2. 1. 1.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[0. 2. 1.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[1. 2. 1.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[2. 2. 1.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[0. 0. 2.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[1. 0. 2.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[2. 0. 2.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[0. 1. 2.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[1. 1. 2.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[2. 1. 2.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[0. 2. 2.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[1. 2. 2.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[2. 2. 2.]
[0. 1. 0.]
[1. 2. 2.]
0.0
[0. 0. 0.]
[1. 1. 0.]
[1. 2. 2.]
0.0
[1. 0. 0.]
[1. 1. 0.]
[1. 2. 2.]
0.0
[2. 0. 0.]
[1. 1. 0.]
[1. 2. 2.]
-0.5
[0. 1. 0.]
[1. 1. 0.]
[1. 2. 2.]
0.0
[1. 1. 0.]
[1. 1. 0.]
[1. 2. 2.]
0.0
[2. 1. 0.]
[1. 1. 0.]
[1. 2. 2.]
0.14285714285714285
[0. 2. 0.]
[1. 1. 0.]
[1. 2. 2.]
0.0
[1. 2. 0.]
[1. 1. 0.]
[1. 2. 2.]

[2. 0. 1.]
[1. 2. 2.]
0.0
[1. 0. 2.]
[2. 0. 1.]
[1. 2. 2.]
0.0
[2. 0. 2.]
[2. 0. 1.]
[1. 2. 2.]
0.0
[0. 1. 2.]
[2. 0. 1.]
[1. 2. 2.]
0.0
[1. 1. 2.]
[2. 0. 1.]
[1. 2. 2.]
0.0
[2. 1. 2.]
[2. 0. 1.]
[1. 2. 2.]
0.0
[0. 2. 2.]
[2. 0. 1.]
[1. 2. 2.]
0.0
[1. 2. 2.]
[2. 0. 1.]
[1. 2. 2.]
0.0
[2. 2. 2.]
[2. 0. 1.]
[1. 2. 2.]
0.0
[0. 0. 0.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[1. 0. 0.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[2. 0. 0.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[0. 1. 0.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[1. 1. 0.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[2. 1. 0.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[0. 2. 0.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[1. 2. 0.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[2. 2. 0.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[0. 0. 1.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[1. 0. 1.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[2. 0. 1.]
[0. 1. 1.]
[1. 2. 2.]
0.2857142857142857
[0. 1. 1.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[1. 1. 1.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[2. 1. 1.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[0. 2. 1.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[1. 2. 1.]
[0. 1. 1.]
[1. 2. 2.]
0.0
[2. 2. 1.]
[0. 1. 1.]
[1. 2. 2.]
0

[2. 1. 2.]
[1. 2. 2.]
0.0
[0. 1. 1.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[1. 1. 1.]
[2. 1. 2.]
[1. 2. 2.]
0.2222222222222222
[2. 1. 1.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[0. 2. 1.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[1. 2. 1.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[2. 2. 1.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[0. 0. 2.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[1. 0. 2.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[2. 0. 2.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[0. 1. 2.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[1. 1. 2.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[2. 1. 2.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[0. 2. 2.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[1. 2. 2.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[2. 2. 2.]
[2. 1. 2.]
[1. 2. 2.]
0.0
[0. 0. 0.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[1. 0. 0.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[2. 0. 0.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[0. 1. 0.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[1. 1. 0.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[2. 1. 0.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[0. 2. 0.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[1. 2. 0.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[2. 2. 0.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[0. 0. 1.]
[0. 2. 2.]
[1. 2. 2.]
0.0
[1. 0. 1.]
[0. 2. 2.]
[1. 2. 2.]
0

[2. 1. 1.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[0. 2. 1.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[1. 2. 1.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[2. 2. 1.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[0. 0. 2.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[1. 0. 2.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[2. 0. 2.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[0. 1. 2.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[1. 1. 2.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[2. 1. 2.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[0. 2. 2.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[1. 2. 2.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[2. 2. 2.]
[2. 2. 0.]
[2. 2. 2.]
0.0
[0. 0. 0.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[1. 0. 0.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[2. 0. 0.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[0. 1. 0.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[1. 1. 0.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[2. 1. 0.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[0. 2. 0.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[1. 2. 0.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[2. 2. 0.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[0. 0. 1.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[1. 0. 1.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[2. 0. 1.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[0. 1. 1.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[1. 1. 1.]
[0. 0. 1.]
[2. 2. 2.]
0.0
[

[2. 0. 0.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[0. 1. 0.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[1. 1. 0.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[2. 1. 0.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[0. 2. 0.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[1. 2. 0.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[2. 2. 0.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[0. 0. 1.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[1. 0. 1.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[2. 0. 1.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[0. 1. 1.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[1. 1. 1.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[2. 1. 1.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[0. 2. 1.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[1. 2. 1.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[2. 2. 1.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[0. 0. 2.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[1. 0. 2.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[2. 0. 2.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[0. 1. 2.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[1. 1. 2.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[2. 1. 2.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[0. 2. 2.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[1. 2. 2.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[2. 2. 2.]
[1. 0. 2.]
[2. 2. 2.]
0.0
[0. 0. 0.]
[2. 0. 2.]
[2. 2. 2.]
0.0
[1. 0. 0.]
[2. 0. 2.]
[2. 2. 2.]
0.0
[

In [326]:
a = [1,2,3,4,5]
p = [0.001, 0.001, 0.001, 0.001, 0.996]
a[np.argmax(np.random.multinomial(1, softmax(p)))]

5